Ερώτημα 1

Αρχικά καταλαβαίνουμε ότι το πρόβλημα μπορεί να επιλυθεί με την μέθοδο του περιπλανώμενου πωλητή, αφού στόχος είναι να περάσουμε από κάθε χρώμα μία μόνο φορά και να επιστρέψουμε στο αρχικό (αφού παράγεται εβδομαδιαία). Ο χρόνος που παίρνει η παραγωγή του κάθε χρώματος δεν έχει σημασία αφού με όποια σειρά και να πάρουμε τα χρώματα θα είναι σταθερός. Άρα έχουμε 40+35+45+32+50 = 202 λεπτά που διαρκεί η διαδικασία παραγωγή και μας ενδιαφέρει να ελαχιστοποιήσουμε τον χρόνο των μεταβάσεων.

Γίνονται import όλες οι βιβλιοθήκες που θα χρησιμοποιήθουν, και δημιουργείτε το μοντέλο μας.

In [1]:
import sys
import math
import random
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
model1 = Model(name = "Ouranio Toxo")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-25


Ο πίνακας φορτώνεται σε ένα data frame για να εκτυπωθεί και από εκεί αποθηκεύονται στο dist όλα τα ζευγάρια των χρόνων που χρείαζονται για παραγωγή j χρώματος όταν προηγούνταν i.

In [2]:
import pandas as pd
data = [[0,11,7,13,11],[5,0,13,15,15],[13,15,0,23,11],[9,13,5,0,3],[3,7,7,7,0]]
df = pd.DataFrame(data, columns=['X1','X2','X3','X4','X5'],index=['X1','X2','X3','X4','X5'])
print (df)
n=df.shape[0]
dist = {(i,j) : df.iloc[i][j] for i in range(n) for j in range(n) if i != j}
print(dist)

    X1  X2  X3  X4  X5
X1   0  11   7  13  11
X2   5   0  13  15  15
X3  13  15   0  23  11
X4   9  13   5   0   3
X5   3   7   7   7   0
{(0, 1): 11, (0, 2): 7, (0, 3): 13, (0, 4): 11, (1, 0): 5, (1, 2): 13, (1, 3): 15, (1, 4): 15, (2, 0): 13, (2, 1): 15, (2, 3): 23, (2, 4): 11, (3, 0): 9, (3, 1): 13, (3, 2): 5, (3, 4): 3, (4, 0): 3, (4, 1): 7, (4, 2): 7, (4, 3): 7}


Εισάγεται η binary μεταβλητή e η οποία για κάθε i,j = 1,2,3,4,5 (0,1,2,3,4 στον κώδικα γιατί η αρίθμηση ξεκινάει από το 0), παίρνει τιμή 1 ή 0, οπού το 1 σημαίνει ότι χρησιμοποιείται αυτή η μετάβαση από i σε j χρώμα, ενώ το 0 ότι δεν χρησιμοποιείται.

In [3]:
vars = model1.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')

Εισάγονται οι περιορισμοί. 
1.Ο πρώτος βεβαιώνει πως το άθροισμα των e σε κάθε γραμμή θα είναι ίσο με 1, που σημαίνει ότι θα μεταβεί σε κάθε χρώμα, μόνο μία φορά.
2.Ο δεύτερος βεβαιώνει πως το άθροισμα των e σε κάθε στήλη θα είναι ίσο με 1, που σημαίνει ότι θα μεταβεί από κάθε χρώμα, μόνο μία φορά.
3.Ο τρίτος περιορισμός βεβεώνει ότι όταν i και j  είναι ίδια το e θα έχει πάντα τιμή 0, αφού δεν έχει νόημα να μεταβούμε από το χρώμα i στο ίδιο χρώμα.

In [4]:
model1.addConstrs(vars.sum(i,'*') == 1 for i in range(n))
model1.addConstrs(vars.sum('*',i) == 1 for i in range(n))
model1.addConstrs(vars.sum(i,i) == 0 for i in range(n))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

Η συνάρτηση αυτή έχει παρθεί έτσι όπως είναι από το documention της Gurobi (https://www.gurobi.com/documentation/9.5/examples/tsp_py.html) με την μόνη διαφορά ότι στην 3η από το τέλος γραμμή έχει προστεθεί το "+ model._vars[j,i]" για να ληφθεί υπόψη η μη συμμετρικότητα του πίνακα. Όπως ήταν γραμμένο ο χρόνος από το i στο j θα ήταν ίδιος με τον χρόνο από το j στο i, κάτι το οποίο στην δική μας περίπτωση προφανώς δεν ίσχυε. Στόχος της συγκεκριμένης συνάρτησης είναι να διαβεβαιώσει ότι δεν θα βρεθεί κάποια διαδρομή που να επιστρέφει το αρχικό χρ΄ώμα χωρίς να έχει περάσει από όλα τα υπόλοιπα προηγουμένως. Αυτό υλοποιείται με την χρήση ενός lazy περιορισμού.

In [5]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        # find the shortest cycle in the selected edge list
        tour = subtour(vals)
        if len(tour) < n:
            # add subtour elimination constr. for every pair of cities in tour
           
            model.cbLazy(quicksum(model._vars[i,j] + model._vars[j,i]
                              for i,j in itertools.combinations(tour, 2))
                     <= len(tour)-1)

Για να επιτευχθεί το παραπάνω είναι απαραίτητη και αυτή η συνάρτηση από το documentation της Gurobi αφού χρησιμοποιείται μέσα στην subtourelim. Ουσιαστικά επιβάλει οι επιλεγμένες αν ακμές να είναι λιγότερες κατά 1 από τους επιλεγμένους κόμβους μέχρι η λίστα των κόμβων που δεν έχουμε επισκευθεί να μείνει άδεια.

In [6]:
def subtour(vals):
    # make a list of edges selected in the solution
    edges = gp.tuplelist((i, j) for i, j in vals.keys()
                         if vals[i, j] > 0.5)
    unvisited = list(range(n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle


Όλα τα δεδομένα μας περνάνε στο μοντέλο και πραγματοποιείται η βελτιστοποίηση.

In [7]:
model1._vars = vars
model1.Params.LazyConstraints = 1
model1.optimize(subtourelim)


vals = model1.getAttr('X', vars)
tour = subtour(vals)

assert len(tour) == n

print('')
print('Optimal tour: %s' % str(tour))
print('Optimal time: %g' % model1.ObjVal)
print('')

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x1f1b5fd4
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 43.0000000
Presolve removed 5 rows and 0 columns
Presolve time: 0.02s
Presolved: 10 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: objective 3.700000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      41.0000000   41.00000  0.00%     -    0s

Cutting p

Όπως φαίνεται από τα αποτελέσματα ο βέλτιστος χρόνος είναι 41 λεπτά. Προσθέτοντας τα 202 που παίρνει η παραγωγή τον χρωμάτων, έχουμε συνολικό ελάχιστο χρόνο 243 λεπτά. Η βέλτιστη διαδρομή είναι 1,4,3,5,2 (προσθέτουμε 1 επειδή η αρίθμηση ξεκινάει από το 0). Γίνεται όμως εύκολα κατανοητό, ότι αφού η παραγωγή είναι εβδομαδιαία και συνεχόμενη θα μπορούσαμε να κρατήσουμε την ίδια αλληλουχία, αλλάζοντας όμως το χρώμα εκκίνησης.
Άρα σωστές απαντήσεις είναι:
1-4-3-5-2
4-3-5-2-1
3-5-2-1-4
5-2-1-4-3
2-1-4-3-5